In [272]:
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
import numpy as np
from datasets import load_dataset, load_metric
import matplotlib.pyplot as plt
import pickle
from sklearn.linear_model import LogisticRegression
from seqeval.metrics import classification_report

In [269]:
def get_embedding(text,tokenizer,model,is_split_into_words=True):
    inputs = tokenizer(text, return_tensors="pt", is_split_into_words=is_split_into_words)
    labels = torch.tensor([1] * inputs["input_ids"].size(1)).unsqueeze(0)  # Batch size 1
    outputs = model(**inputs, labels=labels)
    output = np.array(outputs.hidden_states[12][0].tolist())
    return output

    # back to torch tensor:
    # torch.from_numpy(output).float()

def ht_sm(train_features, train_labels, test_features, classifier, label_idx):
    # aligning target domain to source domain
    sm = torch.nn.Softmax(dim = 0)
    y_prob = [sm(classifier(torch.from_numpy(test_features[i]).float())).tolist()[label_idx] for i in range(len(test_features))]
    y_prob = [(i, val) for i, val in enumerate(y_prob)]
    y_prob = sorted(y_prob, key=lambda x: x[1])
    y_prob_P = [val for val in y_prob if val[1]>0.90]
    y_prob_N = y_prob[:int(len(y_prob_P))]

    sourcePos = [val for i, val in enumerate(train_features) if train_labels[i] == 1]
    sourceNeg = [val for i, val in enumerate(train_features) if train_labels[i] == 0]
    targetPos = [test_features[val[0]] for val in y_prob_P]
    targetNeg = [test_features[val[0]] for val in y_prob_N]
    v = np.mean(sourcePos, axis=0) - np.mean(sourceNeg, axis=0)
    u = np.mean(targetPos, axis=0) - np.mean(targetNeg, axis=0)
    c1 = np.mean(test_features, axis=0)
    c2 = np.mean(np.concatenate([sourcePos, sourceNeg], axis=0), axis=0)

    test_features = hh_lr(u, v, c1, c2, test_features)
    return test_features
    


def ht_lr(train_features, train_labels, test_features):
    # aligning target domain to source domain
    lr_clf = LogisticRegression(max_iter = 10000)
    lr_clf.fit(train_features, train_labels)
    y_pred = lr_clf.predict(test_features)
    y_prob = lr_clf.predict_proba(test_features)[:, 0]
    y_prob = [(i, val, y_pred[i]) for i, val in enumerate(y_prob)]
    y_prob = sorted(y_prob, key=lambda x: x[1])
    y_prob_P = y_prob[:int(len(test_features) / 10)]
    y_prob_N = y_prob[-int(len(test_features) / 10):]

    sourcePos = [val for i, val in enumerate(train_features) if train_labels[i] == 1]
    sourceNeg = [val for i, val in enumerate(train_features) if train_labels[i] == 0]
    targetPos = [test_features[val[0]] for val in y_prob_P]
    targetNeg = [test_features[val[0]] for val in y_prob_N]
    v = np.mean(sourcePos, axis=0) - np.mean(sourceNeg, axis=0)
    u = np.mean(targetPos, axis=0) - np.mean(targetNeg, axis=0)
    c1 = np.mean(test_features, axis=0)
    c2 = np.mean(np.concatenate([sourcePos, sourceNeg], axis=0), axis=0)

    test_features = hh_lr(u, v, c1, c2, test_features)
    return test_features


def hh_lr(u, v, c1, c2, points):
    # household transformation
    u_mag = np.linalg.norm(u)
    u_unit = u / u_mag

    v_mag = np.linalg.norm(v)
    v_unit = v / v_mag

    # Scaling so pos-neg vectors have the same magnitude
    scaled_points = points * v_mag / u_mag
    scaled_c1 = c1 * v_mag / u_mag

    # gettinng dimension of vector space
    k = len(c2)

    # calculating isometric linear transformation: householder transformation
    A = np.eye(k) - (2 * (np.outer(u_unit - v_unit, u_unit - v_unit) / np.inner(u_unit - v_unit, u_unit - v_unit)))

    # applying isometric transformation
    points_after_isometric = scaled_points @ A.T
    c1_after_isometric = scaled_c1 @ A.T

    # translation
    points_after_translation = points_after_isometric + (c2 - c1_after_isometric)

    return points_after_translation

In [3]:
# CoNLL2003 pretrained BERT
tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER", output_hidden_states=True)

In [101]:
# BERT example
example = "European Union, Paris, Berlin"

inputs = tokenizer(example, return_tensors="pt",is_split_into_words=False)
labels = torch.tensor([1] * inputs["input_ids"].size(1)).unsqueeze(0)  # Batch size 1

outputs = model(**inputs, labels=labels)
loss = outputs.loss
logits = outputs.logits 
# equal to model.classifier(outputs.hidden_states[12]) 
# equal to model.classifier(torch.from_numpy(get_embedding("Hello, my dog is cute",tokenizer,model,is_split_into_words=False)).float()) 


#==== ner model ====
nlp = pipeline("ner", model=model, tokenizer=tokenizer)


ner_results = nlp(example)
print(ner_results)

[{'word': 'European', 'score': 0.9995400309562683, 'entity': 'B-ORG', 'index': 1, 'start': 0, 'end': 8}, {'word': 'Union', 'score': 0.9992778897285461, 'entity': 'I-ORG', 'index': 2, 'start': 9, 'end': 14}, {'word': 'Paris', 'score': 0.9994699358940125, 'entity': 'B-LOC', 'index': 4, 'start': 16, 'end': 21}, {'word': 'Berlin', 'score': 0.9994154572486877, 'entity': 'B-LOC', 'index': 6, 'start': 23, 'end': 29}]


In [102]:
tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])

['[CLS]', 'European', 'Union', ',', 'Paris', ',', 'Berlin', '[SEP]']

In [16]:
# load CoNLL2003
datasets = load_dataset("conll2003")
label_list = ["O",'B-MISC', 'I-MISC','B-PER', 'I-PER','B-ORG', 'I-ORG','B-LOC', 'I-LOC']

Reusing dataset conll2003 (C:\Users\zhw027\.cache\huggingface\datasets\conll2003\conll2003\1.0.0\40e7cb6bcc374f7c349c83acd1e9352a4f09474eb691f64f364ee62eb65d0ca6)


In [98]:
label_list

['O', 'B-MISC', 'I-MISC', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']

In [6]:
datasets["train"][0]

{'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0],
 'id': '0',
 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0],
 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7],
 'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.']}

In [40]:
conll_tags_train = [[label_list[i] for i in dict['ner_tags']] for dict in datasets["train"]]
conll_tags_val = [[label_list[i] for i in dict['ner_tags']] for dict in datasets["validation"]]
conll_tags_test = [[label_list[i] for i in dict['ner_tags']] for dict in datasets["test"]]

conll_tokens_train = [dict['tokens'] for dict in datasets["train"]]
conll_tokens_val = [dict['tokens'] for dict in datasets["validation"]]
conll_tokens_test = [dict['tokens'] for dict in datasets["test"]]

In [72]:
conll_emb_train = []
for i,token in enumerate(conll_tokens_train):
    conll_emb_train.append(get_embedding(token,tokenizer,model))
    if i%100 == 0:
        print(i)
        
conll_emb_val = []
for i,token in enumerate(conll_tokens_val):
    conll_emb_val.append(get_embedding(token,tokenizer,model))
    if i%100 == 0:
        print(i)
        
conll_emb_test = []
for i,token in enumerate(conll_tokens_test):
    conll_emb_test.append(get_embedding(token,tokenizer,model))
    if i%100 == 0:
        print(i)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400


In [73]:
conll = {}
conll['train'] = {}
conll['train']['tokens'] = conll_tokens_train
conll['train']['tags'] = conll_tags_train
conll['train']['emb'] = conll_emb_train
conll['val'] = {}
conll['val']['tokens'] = conll_tokens_val
conll['val']['tags'] = conll_tags_val
conll['val']['emb'] = conll_emb_val
conll['test'] = {}
conll['test']['tokens'] = conll_tokens_test
conll['test']['tags'] = conll_tags_test
conll['test']['emb'] = conll_emb_test

In [110]:
# ## Save pickle
# with open("../data/ner/CoNLL2003_IOB.pickle","wb") as fw:
#     pickle.dump(conll, fw)

## Load pickle
with open("../data/ner/CoNLL2003_IOB.pickle","rb") as fr:
    conll = pickle.load(fr)



In [127]:
# example of transforming ndnumpy array to softmax
sm(model.classifier(torch.reshape(torch.from_numpy(conll['train']['emb'][0]).float(), (1, len(conll['train']['emb'][0]), 768))))

tensor([[[9.9986e-01, 3.6812e-05, 2.4579e-05, 9.2055e-06, 8.1857e-06,
          2.2269e-05, 1.4744e-05, 6.9506e-06, 1.3782e-05],
         [1.4827e-04, 2.5760e-04, 3.9912e-05, 8.2197e-05, 1.5342e-05,
          9.9934e-01, 3.7962e-05, 6.1841e-05, 1.5613e-05],
         [9.9995e-01, 8.1573e-06, 7.9834e-06, 2.9079e-06, 2.6201e-06,
          6.7544e-06, 1.2999e-05, 1.9078e-06, 4.5467e-06],
         [5.0466e-05, 9.9968e-01, 1.6745e-04, 1.7678e-05, 2.0535e-05,
          2.9858e-05, 1.1429e-05, 1.3439e-05, 1.0506e-05],
         [9.9995e-01, 7.1225e-06, 1.3930e-05, 3.3021e-06, 2.7692e-06,
          3.7986e-06, 7.8546e-06, 1.8355e-06, 5.0073e-06],
         [9.9997e-01, 5.1965e-06, 5.2533e-06, 2.1644e-06, 1.9781e-06,
          2.3487e-06, 4.4365e-06, 1.4817e-06, 3.1004e-06],
         [9.9997e-01, 6.6605e-06, 5.5180e-06, 2.1587e-06, 1.9227e-06,
          2.9116e-06, 4.4655e-06, 1.4905e-06, 3.5648e-06],
         [4.0992e-05, 9.9961e-01, 2.3095e-04, 2.0966e-05, 1.9953e-05,
          4.0297e-05, 9.310

In [133]:
# example of transforming ndnumpy array to softmax, but only extracting one token
sm(model.classifier(torch.reshape(torch.from_numpy(conll['train']['emb'][0][1]).float(), (1, 1, 768))))

tensor([[[1.4827e-04, 2.5760e-04, 3.9912e-05, 8.2197e-05, 1.5342e-05,
          9.9934e-01, 3.7962e-05, 6.1841e-05, 1.5613e-05]]],
       grad_fn=<SoftmaxBackward>)

# load tech_test data

In [134]:
len([y for x in conll['train']['emb'] for y in x])

300677

In [146]:
tech_test = {}
tech_test['test'] = {}
tech_test['test']['tokens'] = []
tech_test['test']['tags'] = []
tech_test['test']['emb'] = []

with open('../data/ner/tech_test.txt', 'r') as f:
    tokens = []
    tags = []
    for line in f.readlines():
        if line == '\n':
            tech_test['test']['tokens'].append(tokens)
            tech_test['test']['tags'].append(tags)
            tokens = []
            tags = []
        else:
            if line.split(' ')[0] != '':
                tokens.append(line.split(' ')[0])
                tag = line.split(' ')[1][:-1]
                if tag[:1] == 'E':
                    tag = 'I'+tag[1:]
                elif tag[:1] == 'S':
                    tag = 'B'+tag[1:]
                tags.append(tag)

In [147]:
for i,token in enumerate(tech_test['test']['tokens']):
    tech_test['test']['emb'].append(get_embedding(token,tokenizer,model))
    if i%100 == 0:
        print(i)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900


In [136]:
# ## Save pickle
# with open("../data/ner/tech_test_IOB.pickle","wb") as fw:
#     pickle.dump(tech_test, fw)

## Load pickle
with open("../data/ner/tech_test_IOB.pickle","rb") as fr:
    tech_test = pickle.load(fr)

# Householder Transformation

In [144]:
label_list

['O', 'B-MISC', 'I-MISC', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']

In [ ]:
# word-token mapping

train_data = conll['train']
val_data = conll['test']
test_data = tech_test['test']


# preprocessing
## train all subtokens
train_all_subtokens = []
train_all_tokens = []
train_all_tags = []
train_subtoken_map = []
subtoken_length = 0
for sent_idx, tokens in enumerate(train_data['tokens']):
    train_all_tokens += tokens
    train_all_tags += train_data['tags'][sent_idx]
    subtoken_length += 1
    sent_subtoken = []
    for token in tokens:
        subtokens = tokenizer([token], return_tensors="pt",is_split_into_words=True)["input_ids"][0]
        subtokens = tokenizer.convert_ids_to_tokens(subtokens)
        train_subtoken_map.append(subtoken_length)
        subtoken_length += len(subtokens[1:-1])
        sent_subtoken += subtokens[1:-1]
    subtoken_length += 1
    train_all_subtokens += ['[CLS]'] + sent_subtoken + ['[SEP]']

## train all emb
train_all_emb = [token_emb for sent in train_data['emb'] for token_emb in sent]

# preprocessing
## val all subtokens
val_all_subtokens = []
val_all_tokens = []
val_all_tags = []
val_subtoken_map = []
subtoken_length = 0
for sent_idx, tokens in enumerate(val_data['tokens']):
    val_all_tokens += tokens
    val_all_tags += val_data['tags'][sent_idx]
    subtoken_length += 1
    sent_subtoken = []
    for token in tokens:
        subtokens = tokenizer([token], return_tensors="pt",is_split_into_words=True)["input_ids"][0]
        subtokens = tokenizer.convert_ids_to_tokens(subtokens)
        val_subtoken_map.append(subtoken_length)
        subtoken_length += len(subtokens[1:-1])
        sent_subtoken += subtokens[1:-1]
    subtoken_length += 1
    val_all_subtokens += ['[CLS]'] + sent_subtoken + ['[SEP]']

## val all emb
val_all_emb = [token_emb for sent in val_data['emb'] for token_emb in sent]

    
## test all subtokens
test_all_subtokens = []
test_all_tokens = []
test_all_tags = []
test_subtoken_map = []
subtoken_length = 0
for sent_idx, tokens in enumerate(test_data['tokens']):
    test_all_tokens += tokens
    test_all_tags += test_data['tags'][sent_idx]
    subtoken_length += 1
    sent_subtoken = []
    for token in tokens:
        subtokens = tokenizer([token], return_tensors="pt",is_split_into_words=True)["input_ids"][0]
        subtokens = tokenizer.convert_ids_to_tokens(subtokens)
        test_subtoken_map.append(subtoken_length)
        subtoken_length += len(subtokens[1:-1])
        sent_subtoken += subtokens[1:-1]
    subtoken_length += 1
    test_all_subtokens += ['[CLS]'] + sent_subtoken + ['[SEP]']

## test all emb
test_all_emb = [token_emb for sent in test_data['emb'] for token_emb in sent]

In [270]:
# HT

test_features_ht = {}


# divide label by label
# map between token and subtoken embedding
for label_idx, target_label in enumerate(label_list):
    # divide and map source data
    train_features = []
    train_labels = []
    for tok_idx, tag in enumerate(train_all_tokens):
        map_idx = train_subtoken_map[tok_idx]
        emb = train_all_emb[map_idx]
        tag = train_all_tags[tok_idx]
        train_features.append(emb)
        if tag == target_label:
            train_labels.append(1)
        else:
            train_labels.append(0)
 
    # divide and map target data
    test_features = []
    test_labels = []
    for tok_idx, tag in enumerate(test_all_tokens):
        map_idx = test_subtoken_map[tok_idx]
        emb = test_all_emb[map_idx]
        tag = test_all_tags[tok_idx]
        test_features.append(emb)
        if tag == target_label:
            test_labels.append(1)
        else:
            test_labels.append(0)

    # transform to numpy
    train_features = np.array(train_features)
    train_labels = np.array(train_labels)
    test_features = np.array(test_features)

    # householder transformation
    test_features_ht[target_label] = ht_sm(train_features, train_labels, test_features, model.classifier, label_idx)
    print(target_label)

O
B-MISC
I-MISC
B-PER
I-PER
B-ORG
I-ORG
B-LOC
I-LOC


In [260]:
# only ht OOV tokens
OOV = list(set([token for token in test_all_tokens if token not in train_all_tokens]))
for target_label in label_list:
    OOV_ht = []
    for token_idx, emb in enumerate(test_features_ht[target_label]):
        if test_all_tokens[token_idx] in OOV:
            OOV_ht.append(emb)
        else:
            OOV_ht.append(test_features[token_idx])
    OOV_ht = np.array(OOV_ht)
    test_features_ht[target_label] = OOV_ht

In [146]:
print(len(train_features))
print(len(train_labels))
print(len(test_features))
print(len(test_labels))

203621
203621
54070
54070


In [147]:
## Save pickle
with open("../data/ner/test_features_ht_IOB.pickle","wb") as fw:
    pickle.dump(test_features_ht, fw)

# ## Load pickle
# with open("../data/ner/test_features_ht_IOB.pickle","rb") as fr:
#     test_features_ht = pickle.load(fr)

# label prediction

In [266]:
# with ht
prediction_ht = []
for index in range(len(test_features_ht['O'])):
    tag_prob_layer = []
    for tag_index, target_label in enumerate(label_list):
        token_emb = test_features_ht[target_label][index] # get ht-ed token embedding
        token_emb = torch.from_numpy(token_emb).float() # make it as torch
        logit = model.classifier(token_emb) # pass emb into finetuned classifier to get logit
        sm = torch.nn.Softmax(dim = 0) # pass logit into softmax layer
        softmax = sm(logit)
        tag_prob_layer.append(softmax.tolist()[tag_index]) # only take probability of target tag
#         tag_prob_layer.append(logit.tolist()[tag_index]) # only take logit of target tag
#     tag_prob_layer = torch.from_numpy(np.array(tag_prob_layer)).float() # transform to tensor
#     sm = torch.nn.Softmax(dim = 0)
#     output_layer = sm(tag_prob_layer) # pass logits to softmax
#     output_layer = output_layer.tolist()
#     predict_tag_index = np.argmax(output_layer)
    predict_tag_index = np.argmax(tag_prob_layer)
    predict_tag = label_list[predict_tag_index]
    prediction_ht.append(predict_tag)
print(classification_report(y_true=test_all_tags, y_pred=prediction_ht, labels=[label for label in label_list if label != "O"]))

              precision    recall  f1-score   support

      B-MISC       0.38      0.49      0.43       365
      I-MISC       0.25      0.50      0.34       132
       B-PER       0.93      0.93      0.93      1091
       I-PER       0.96      0.98      0.97       568
       B-ORG       0.69      0.83      0.75       872
       I-ORG       0.54      0.77      0.63       476
       B-LOC       0.65      0.85      0.74       489
       I-LOC       0.48      0.77      0.59       140

   micro avg       0.69      0.83      0.75      4133
   macro avg       0.61      0.76      0.67      4133
weighted avg       0.72      0.83      0.77      4133



In [273]:
print(classification_report([test_all_tags], [prediction_ht]))

              precision    recall  f1-score   support

         LOC       0.60      0.80      0.69       489
        MISC       0.34      0.47      0.40       365
         ORG       0.64      0.79      0.71       873
         PER       0.93      0.93      0.93      1094

   micro avg       0.68      0.80      0.74      2821
   macro avg       0.63      0.75      0.68      2821
weighted avg       0.71      0.80      0.75      2821



In [274]:
# without ht
tech_test_emb = [test_all_emb[i] for i in test_subtoken_map]
prediction = []
for index in range(len(tech_test_emb)):
    token_emb = tech_test_emb[index] # get ht-ed token embedding
    token_emb = torch.from_numpy(token_emb).float() # make it as torch
    logit = model.classifier(token_emb) # pass emb into finetuned classifier to get logit
    sm = torch.nn.Softmax(dim = 0) # pass logit into softmax layer
    softmax = sm(logit)
    predict_tag_index = np.argmax(softmax.tolist())
    predict_tag = label_list[predict_tag_index]
    prediction.append(predict_tag)

In [275]:
print(classification_report([test_all_tags], [prediction]))

              precision    recall  f1-score   support

         LOC       0.60      0.80      0.69       489
        MISC       0.34      0.47      0.40       365
         ORG       0.64      0.78      0.70       873
         PER       0.93      0.92      0.92      1094

   micro avg       0.68      0.80      0.74      2821
   macro avg       0.63      0.75      0.68      2821
weighted avg       0.71      0.80      0.75      2821



In [268]:
print(classification_report(y_true=test_all_tags, y_pred=prediction, labels=[label for label in label_list if label != "O"]))

              precision    recall  f1-score   support

      B-MISC       0.38      0.49      0.43       365
      I-MISC       0.26      0.50      0.34       132
       B-PER       0.93      0.93      0.93      1091
       I-PER       0.96      0.98      0.97       568
       B-ORG       0.68      0.82      0.75       872
       I-ORG       0.53      0.77      0.63       476
       B-LOC       0.65      0.85      0.74       489
       I-LOC       0.48      0.76      0.59       140

   micro avg       0.69      0.83      0.75      4133
   macro avg       0.61      0.76      0.67      4133
weighted avg       0.72      0.83      0.76      4133



In [167]:
# conll test
conll_val_emb = [val_all_emb[i] for i in val_subtoken_map]
prediction = []
for index in range(len(conll_val_emb)):
    token_emb = conll_val_emb[index] # get ht-ed token embedding
    token_emb = torch.from_numpy(token_emb).float() # make it as torch
    logit = model.classifier(token_emb) # pass emb into finetuned classifier to get logit
    sm = torch.nn.Softmax(dim = 0) # pass logit into softmax layer
    softmax = sm(logit)
    predict_tag_index = np.argmax(softmax.tolist())
    predict_tag = label_list[predict_tag_index]
    prediction.append(predict_tag)
    if index%10000 == 0:
        print(index)

0
10000
20000
30000
40000


In [173]:
print(classification_report(y_true=val_all_tags, y_pred=prediction, labels=label_list))

              precision    recall  f1-score   support

           O       1.00      0.99      0.99     38323
      B-MISC       0.83      0.85      0.84       702
      I-MISC       0.65      0.76      0.70       216
       B-PER       0.96      0.96      0.96      1617
       I-PER       0.99      0.99      0.99      1156
       B-ORG       0.91      0.93      0.92      1661
       I-ORG       0.89      0.92      0.91       835
       B-LOC       0.94      0.93      0.94      1668
       I-LOC       0.88      0.90      0.89       257

    accuracy                           0.98     46435
   macro avg       0.89      0.92      0.90     46435
weighted avg       0.98      0.98      0.98     46435

